This file will generate the required data in the following data tree format:
1. Final_data(dir) :
  - ground_truth.csv
  - input_para1.csv
  - input_question.csv
  - theme_interval.csv
2. finaldataprocessed.csv(file)
3. synthetic_data.csv(file)

## **Imports**

In [ ]:
# Module Installations

!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00


In [ ]:
# Module Imports
import os
import gdown
import random
import numpy as np
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, AutoModelForQuestionAnswering, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM

## **File Generation**

In [ ]:
import pandas as pd
para_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1GNHhH81J1pEZSB-OSGRTtqIUQCCNhJ6I')
question_ans = pd.read_csv('https://drive.google.com/uc?export=download&id=1GQ3-E7k60K16f47A18ptQfKUBkhxQudM')

In [ ]:
df = question_ans
df2 = para_data

In [ ]:
df

,question,theme,paragraph_id,answer
0,Which company produces the iPod?,IPod,1,Apple
1,When was the original iPod released?,IPod,1,"October 23, 2001"
2,How many different types of iPod are currently...,IPod,1,three
3,What kind of device is the iPod?,IPod,1,portable media players
4,The iPod Touch uses what kind of interface?,IPod,1,touchscreen
...,...,...,...,...
939,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,ballads
940,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,hot AC
941,What AC format is especially known for playing...,Adult_contemporary_music,642,soft AC
942,When is the later of the two times that Christ...,Adult_contemporary_music,643,New Year's Day


In [ ]:
df2

,id,paragraph,theme
0,1,The iPod is a line of portable media players a...,IPod
1,2,"Like other digital music players, iPods can se...",IPod
2,3,Apple's iTunes software (and other alternative...,IPod
3,4,"Before the release of iOS 5, the iPod branding...",IPod
4,5,"In mid-2015, a new model of the iPod Touch was...",IPod
...,...,...,...
1174,1175,Adding new Customers# Customer records offer a...,DevRev
1175,1176,Apart from customer records that get automatic...,DevRev
1176,1177,Authentication to DevRev APIs requires a perso...,DevRev
1177,1178,Generate a Personal Access Token (PAT)# In the...,DevRev


In [ ]:
para_id_list = df.paragraph_id.tolist()
paras = []
for i in para_id_list:
    paras.append(df2['paragraph'][df2.id==i].tolist()[0])
df['paragraph'] = paras
df.rename(columns = {'question':'Question', 'theme':'Theme','answer':'Answer_text', 'paragraph':'Paragraph'}, inplace = True)
poss = []
for i in range(len(df)):
    poss.append('TRUE')
df['Answer_possible'] = poss
start = []
for i in range(len(df)):
  # print(df['Answer_text'][i])
  s = str(df['Answer_text'][i])
  start.append([df['Paragraph'][i].find(s)])
df['Answer_start'] = start
answers = df['Answer_text'].tolist()
ans = []
for i in answers:
    ans.append([i])
df['Answer_text'] = ans
df.to_csv('/content/finaldataprocessed.csv')

In [ ]:
df = pd.read_csv('/content/finaldataprocessed.csv')
df

,Unnamed: 0,Question,Theme,paragraph_id,Answer_text,Paragraph,Answer_possible,Answer_start
0,0,Which company produces the iPod?,IPod,1,['Apple'],The iPod is a line of portable media players a...,True,[105]
1,1,When was the original iPod released?,IPod,1,"['October 23, 2001']",The iPod is a line of portable media players a...,True,[147]
2,2,How many different types of iPod are currently...,IPod,1,['three'],The iPod is a line of portable media players a...,True,[302]
3,3,What kind of device is the iPod?,IPod,1,['portable media players'],The iPod is a line of portable media players a...,True,[22]
4,4,The iPod Touch uses what kind of interface?,IPod,1,['touchscreen'],The iPod is a line of portable media players a...,True,[400]
...,...,...,...,...,...,...,...,...
939,939,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,['ballads'],"In the 1980s, lush jazz-R&B fusion (George Ben...",True,[318]
940,940,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,['hot AC'],Contemporary Christian music (CCM) has several...,True,[202]
941,941,What AC format is especially known for playing...,Adult_contemporary_music,642,['soft AC'],In recent years it has become common for many ...,True,[72]
942,942,When is the later of the two times that Christ...,Adult_contemporary_music,643,"[""New Year's Day""]",These Christmas music marathons typically star...,True,[143]


In [ ]:
df = df.rename(columns={'Unnamed: 0':'id'})
df['id'] =df['id']+1
df.to_csv('/content/finaldataprocessed.csv')

In [ ]:
os.mkdir('/content/Final_data')

In [ ]:
m='/content/Final_data'
df = pd.read_csv('/content/finaldataprocessed.csv')

In [ ]:
df

,Unnamed: 0,id,Question,Theme,paragraph_id,Answer_text,Paragraph,Answer_possible,Answer_start
0,0,1,Which company produces the iPod?,IPod,1,['Apple'],The iPod is a line of portable media players a...,True,[105]
1,1,2,When was the original iPod released?,IPod,1,"['October 23, 2001']",The iPod is a line of portable media players a...,True,[147]
2,2,3,How many different types of iPod are currently...,IPod,1,['three'],The iPod is a line of portable media players a...,True,[302]
3,3,4,What kind of device is the iPod?,IPod,1,['portable media players'],The iPod is a line of portable media players a...,True,[22]
4,4,5,The iPod Touch uses what kind of interface?,IPod,1,['touchscreen'],The iPod is a line of portable media players a...,True,[400]
...,...,...,...,...,...,...,...,...,...
939,939,940,What type of songs was Leona Lewis known for?,Adult_contemporary_music,640,['ballads'],"In the 1980s, lush jazz-R&B fusion (George Ben...",True,[318]
940,940,941,"Along with mainstream AC, what format has also...",Adult_contemporary_music,641,['hot AC'],Contemporary Christian music (CCM) has several...,True,[202]
941,941,942,What AC format is especially known for playing...,Adult_contemporary_music,642,['soft AC'],In recent years it has become common for many ...,True,[72]
942,942,943,When is the later of the two times that Christ...,Adult_contemporary_music,643,"[""New Year's Day""]",These Christmas music marathons typically star...,True,[143]


In [ ]:
paras = df.Paragraph.unique()

In [ ]:
df[df["Paragraph"]==paras[0]].Theme[0]
df = df.rename(columns={'Answer_Text':'Answer_text'})

In [ ]:
data = []
for i, para in enumerate(paras):
    data_dict = {}
    data_dict['id']=i+1
    data_dict['paragraph']=para
    data_dict['theme'] = df[df["Paragraph"]==para].iloc[0].Theme
    data.append(data_dict)

In [ ]:
df2 = pd.DataFrame(data)

In [ ]:
df_theme=df2.copy()
df2.to_csv(m+'/input_para1.csv', header=True, index=False)

In [ ]:
data = []
for i in range(len(df.Question)):
    data_dict = dict()
    data_dict['id'] = i+1
    data_dict['question'] = df.Question[i]
    data_dict['theme'] = df.Theme[i]
    data.append(data_dict)

In [ ]:
df2 = pd.DataFrame(data)
df2.to_csv(m+'/input_question.csv', header=True, index=False)

In [ ]:
data = []
for i in range(len(df.Question)):
    data_dict = dict()
    data_dict['question_id'] = i+1

    # if(df.Answer_start[i]=='[]'):
    #  data_dict['paragraph_id'] = -1
    # else:
    data_dict['paragraph_id']=df_theme[df_theme.paragraph==df.Paragraph[i]].index.tolist()[0]+1
    data_dict['answers'] = df.Answer_text[i]
    data.append(data_dict)
df2 = pd.DataFrame(data)
df2.to_csv(m+'/ground_truth.csv', header=True, index=False)

In [ ]:
data = []
for i in df.Theme.unique():
    data_dict = dict()
    start, end = (df.index[df['Theme']==i][[0,-1]]+1).tolist()
    data_dict["theme"] = i
    data_dict["start"] = start
    data_dict['end'] = end
    data.append(data_dict)
df2 = pd.DataFrame(data)
df2.to_csv(m+'/theme_interval.csv', header=True, index=False)

## **Model Imports**

In [ ]:
# Question Answering Model
qa_model_name = "sultan/BioM-ELECTRA-Large-SQuAD2"
qa = pipeline('question-answering', model=qa_model_name)

In [ ]:
# Question Generation Model
qg_model_name = "wiselinjayajos/t5-end2end-questions-generation"
tokenizer = AutoTokenizer.from_pretrained(qg_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(qg_model_name)

In [ ]:
# Backtranslating Models for Data Augmentation and Paraphrasing

# English to French Translator
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'
first_model_tkn = MarianTokenizer.from_pretrained(first_model_name)
first_model = MarianMTModel.from_pretrained(first_model_name)

# French to English Translator
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'
second_model_tkn = MarianTokenizer.from_pretrained(second_model_name)
second_model = MarianMTModel.from_pretrained(second_model_name)

/usr/local/lib/python3.8/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## **Functions**

In [ ]:
def quesGen(input_string,tokenizer=tokenizer,model=model):
    generator_args = {"max_length": 5000,"num_beams": 4,"length_penalty": 1.5,"no_repeat_ngram_size": 3,"early_stopping": True,}
    input_string = "generate questions: " + input_string + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = output[0]
    output = output.split("? ")
    for ques_index in range(len(output)):
        ques = output[ques_index]
        if len(ques)<7:
            output.remove(output[ques_index])
        elif ques[-1]!='?':
            ques = ques +'?'
            output[ques_index] = ques
    return output

def format_batch_texts(language_code, batch_texts):
    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]
    return formated_bach
    
def perform_translation(batch_texts, model, tokenizer, language="fr"):
    formated_batch_texts = format_batch_texts(language, batch_texts)
    translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))
    translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    return translated_texts

## **Dataset Creation**

Answerable Questions Data Generation

In [ ]:
df = pd.read_csv('/content/finaldataprocessed.csv')
paragraphs = df.Paragraph.unique().tolist()

In [ ]:
# Main Code Block for Synthetic Data Generation
data = dict()
questions = []
themes = []
contexts = []
answers = []
start = []
end = []
poss = []

for para in paragraphs:
    try:
        theme = df[df.Paragraph == para].Theme.iloc[0]
        # Performing English to French Translation
        translated_texts = perform_translation([para], first_model, first_model_tkn)
        # Performing French to English Translation
        back_translated_texts = perform_translation(translated_texts, second_model, second_model_tkn)
        para_new = back_translated_texts[0]
        # Question Generation
        question = quesGen(para_new)
        for ques in question:
          # QA on Generated Questions
          result = qa(question=ques, context=para_new)
          questions.append(ques)
          themes.append(theme)
          contexts.append(para_new)
          answers.append([result['answer']])
          start.append([result['start']])
          end.append([result['end']])
          poss.append('TRUE')
    except:
        pass

data['Paragraph'] = contexts
data['Question'] = questions
data['Theme'] = themes
data['Answer_text'] = answers
data['Answer_start'] = start
data['Answer_end'] = end
data['Answer_possible'] = poss
new_df = pd.DataFrame(data)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Unanswerable Question Generation

In [ ]:
# Theme wise creating random unanswerable questions and answers
themes = new_df.Theme.unique().tolist()
count = len(new_df)
for i in themes:
    questions = new_df[new_df.Theme==i]['Question'].tolist()
    false_ques_no = len(questions)//3
    for j in range(false_ques_no):
        paragraphs = new_df[new_df.Theme==i]['Paragraph'].unique().tolist()
        question = random.choice(questions)
        questions.remove(question)
        ans_para =new_df[new_df.Question==question]['Paragraph'].tolist()[0]
        context = random.choice(paragraphs)
        new_df.loc[count] = [context,question,i,[],[],[],'FALSE']
        count+=1

In [ ]:
# Shuffling the dataset
df = new_df.sample(frac=1)

In [ ]:
# saving the dataset with synthetically generated data
df.to_csv(f'synthetic_data.csv')